In [ ]:
"""
    Pappus' creativity is due to him (implicitly) distinguishing between numerical identity and conceptual identity
    in step 1 and applying this distinction to the triangle ABC to take advantage of Euclid I.4.
"""

!python --version

## install and import packages
# !pip install nbimporter # to import notebooks
# !pip install rdflib

## import modules
import copy
# import glob
import json
# import os
# import rdflib
import requests
import typing # to add advanced type hinting

## import notebooks
import sparql_classes
import sparql_queries

# !pwd

# PROOFS OF EUCLID I.5

In [ ]:
# %%time
################################
## FIND INFORMATION CONCERCING 
## THE CONCEPTUAL SPACE OF 
## A GIVEN PROOF STEP
################################

## select a proof step
# proof_step_iri:str = "<https://www.foom.com/pappus_proofAristotle#00000000000000000014>"
proof_step_analysis:dict = {"proof_step_iri": "<https://www.foom.com/pappus_proofAristotle#00000000000000000014>" }

# find label of proof step
def query_find_proof_step_label(proof_step_iri:str) -> str:
    sparql_query:str = f"""
        SELECT DISTINCT 
            ?proof_step_label
        WHERE {{
            {proof_step_iri} 
                rdfs:label ?proof_step_label .
        }}
    """
    return sparql_query

def find_proof_step_label(proof_step_analysis:dict,
                          sparql_query_text:str):
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        proof_step_analysis["proof_step_label"] = f"{row.proof_step_label}"
    return proof_step_analysis

proof_step_analysis:dict = find_proof_step_label(proof_step_analysis,
                                                 sparql_queries.query_find_proof_step_label(proof_step_analysis["proof_step_iri"])
                                                )

# find reification values of proof step
def find_reification_values(statement_iri:str):
    sparql_query_text:str = sparql_queries.query_find_reification_values(statement_iri)
    reification_values:dict = {}
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        reification_values:dict = {
            "subject_iri": f"<{row.subject_iri}>",
            "subject_label": f"<{row.subject_label}>",
            "predicate_iri": f"<{row.predicate_iri}>",
            "predicate_label": f"<{row.predicate_label}>",
            "object_iri": f"<{row.object_iri}>",
            "object_label": f"<{row.object_label}>"
        }
    return reification_values

proof_step_analysis["proof_step_reification_values"] = find_reification_values(proof_step_analysis["proof_step_iri"])

# find the proof to which the given proof step belongs
def find_proof(proof_step_analysis:dict,
               sparql_query_text:str):
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        proof_step_analysis["proof_iri"] = f"<{row.proof_iri}>"
        proof_step_analysis["proof_label"] = f"<{row.proof_label}>"
    return proof_step_analysis

proof_step_analysis:dict = find_proof(proof_step_analysis,
                                      sparql_queries.query_find_proof(proof_step_analysis["proof_step_iri"])
                                     )

## find information concerning the proof:
# 1. goal
# 2. conceptual items connected to the goal
# 3. statement to prove
# 4. conceptual items connected to the statement to prove
# 5. source that contains the proof
# 6. author of the proof
# 7. remaining conceptual space of the proof

# goal
def find_goal_of_proof(proof_step_analysis:dict,
                       sparql_query_text:str):
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        proof_step_analysis["proof_goal_iri"] = f"<{row.goal_iri}>"
        proof_step_analysis["proof_goal_label"] = f"<{row.goal_label}>"
    return proof_step_analysis

proof_step_analysis:dict = find_goal_of_proof(proof_step_analysis,
                                              sparql_queries.query_find_goals_of_proof(proof_step_analysis["proof_iri"])
                                             )

proof_step_analysis["proof_goal_reification_values"] = find_reification_values(proof_step_analysis["proof_goal_iri"])
    
## find preceding proofs steps
def find_antecedent_proof_steps(proof_step_analysis:dict,
                                results_query:str):
    proof_step_analysis["antencedent_proof_steps"] = {f"<{row.antecedent_iri}>":{} for row in sparql_classes.SparqlQueryResults(results_query)}
    return proof_step_analysis

proof_step_analysis:dict = find_antecedent_proof_steps(proof_step_analysis,
                                                       sparql_queries.query_find_antecedent_proof_steps(
                                                           proof_step_analysis["proof_step_iri"],
                                                           proof_step_analysis["proof_iri"]
                                                       )
                                                      )

# find reification values for preceding proof steps
def find_reification_values_for_antecedent_proof_steps(proof_step_analysis:dict):
    for antencedent_step_iri in proof_step_analysis["antencedent_proof_steps"]:
        proof_step_analysis["antencedent_proof_steps"][antencedent_step_iri]["reification_values"] = find_reification_values(antencedent_step_iri)
    return proof_step_analysis

proof_step_analysis:dict = find_reification_values_for_antecedent_proof_steps(proof_step_analysis)

## find the conceptual space of the preceding proof steps
def find_related_objects(proof_step_analysis:dict):
    proof_steps:set = {proof_step for proof_step in proof_step_analysis["antencedent_proof_steps"]}
    proof_steps.add(proof_step_analysis["proof_step_iri"])
    proof_step_analysis["related_objects"] = {proof_step_iri: {
                                                    "iris": [],
                                                    "labels": []
                                                } for proof_step_iri in proof_steps
                                            }
    for proof_step_iri in proof_steps:
        sparql_query_text:str = sparql_queries.find_related_objects(proof_step_iri)
        for row in sparql_classes.SparqlQueryResults(sparql_query_text):
            proof_step_analysis["related_objects"][proof_step_iri]["iris"].append(row.object_iri)
            proof_step_analysis["related_objects"][proof_step_iri]["labels"].append(row.object_label)
    return proof_step_analysis


def find_mathematically_directly_related_items(proof_step_analysis:dict,
                                  statement_iri:str) -> dict:
    sparql_query_text:str = sparql_queries.query_find_mathematically_directly_related_items(statement_iri)
    proof_step_analysis["mathematically_directly_related_items"][statement_iri] = {
                "iris": [],
                "labels": []
            }
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        proof_step_analysis["mathematically_directly_related_items"][statement_iri]["iris"].append(row.directly_related_iri)
        proof_step_analysis["mathematically_directly_related_items"][statement_iri]["labels"].append(row.directly_related_label)
    return proof_step_analysis

def find_mathematically_indirectly_related_items(proof_step_analysis:dict,
                                  statement_iri:str) -> dict:
    sparql_query_text:str = sparql_queries.query_find_mathematically_indirectly_related_items(statement_iri)
    proof_step_analysis["mathematically_indirectly_related_items"][statement_iri] = {
                "iris": [],
                "labels": []
            }
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        proof_step_analysis["mathematically_indirectly_related_items"][statement_iri]["iris"].append(row.indirectly_related_iri)
        proof_step_analysis["mathematically_indirectly_related_items"][statement_iri]["labels"].append(row.indirectly_related_label)
    return proof_step_analysis

def find_statement_iris(proof_step_analysis:dict) -> set:
    """
    Recursively find and return all values associated with keys ending with '_iri' in the dictionary and its nested dictionaries.

    Args:
        proof_step_analysis (dict): The dictionary to search through.

    Returns:
        statement_iris: A list of values associated with keys ending with '_iri'.

    # Example usage:
    # test_dict = {
    #     "a": 1,
    #     "b": {"b_iri": "value1", "b2": {"b2_iri": "value2"}},
    #     "c": {"c1": {"c1_iri": "value3"}, "c2": 5}
    # }
    # result = find_statement_iris(test_dict)
    # print(result)  # Output should be {'value1', 'value2', 'value3'}
    """
    statement_iris = set()
    def recursive_search(dictionary: dict):
        for key, value in dictionary.items():
            if key.endswith("_iri"):
                statement_iris.add(value)
            elif isinstance(value, dict):
                recursive_search(value)

    recursive_search(proof_step_analysis)
    return statement_iris

def find_mathematically_related_items(proof_step_analysis:dict):
    statement_iris:set = find_statement_iris(proof_step_analysis)
    proof_step_analysis["mathematically_directly_related_items"] = {}
    proof_step_analysis["mathematically_indirectly_related_items"] = {}
    for statement_iri in statement_iris:
        proof_step_analysis = find_mathematically_directly_related_items(proof_step_analysis, statement_iri)
        proof_step_analysis = find_mathematically_indirectly_related_items(proof_step_analysis, statement_iri)
    return proof_step_analysis
    
proof_step_analysis:dict = find_mathematically_related_items(proof_step_analysis)

def find_conceptually_directly_related_items(proof_step_analysis:dict,
                                  statement_iri:str) -> dict:
    sparql_query_text:str = sparql_queries.query_find_conceptually_directly_related_items(statement_iri)
    proof_step_analysis["conceptually_directly_related_items"][statement_iri] = {
                "iris": [],
                "labels": []
            }
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        proof_step_analysis["conceptually_directly_related_items"][statement_iri]["iris"].append(row.directly_related_iri)
        proof_step_analysis["conceptually_directly_related_items"][statement_iri]["labels"].append(row.directly_related_label)
    return proof_step_analysis

def find_conceptually_indirectly_related_items(proof_step_analysis:dict,
                                  statement_iri:str) -> dict:
    sparql_query_text:str = sparql_queries.query_find_conceptually_indirectly_related_items(statement_iri)
    proof_step_analysis["conceptually_indirectly_related_items"][statement_iri] = {
                "iris": [],
                "labels": []
            }
    for row in sparql_classes.SparqlQueryResults(sparql_query_text):
        proof_step_analysis["conceptually_indirectly_related_items"][statement_iri]["iris"].append(row.indirectly_related_iri)
        proof_step_analysis["conceptually_indirectly_related_items"][statement_iri]["labels"].append(row.indirectly_related_label)
    return proof_step_analysis

def find_conceptually_related_items(proof_step_analysis:dict):
    statement_iris:set = find_statement_iris(proof_step_analysis)
    proof_step_analysis["conceptually_directly_related_items"] = {}
    proof_step_analysis["conceptually_indirectly_related_items"] = {}
    for statement_iri in statement_iris:
        proof_step_analysis = find_conceptually_directly_related_items(proof_step_analysis, statement_iri)
        proof_step_analysis = find_conceptually_indirectly_related_items(proof_step_analysis, statement_iri)
    return proof_step_analysis

proof_step_analysis:dict = find_conceptually_related_items(proof_step_analysis)


## find analogous proofs 

## find analogous proof parts

## find other proofs or proof parts connected to the given proof step or proof


proof_step_analysis

In [ ]:
for key in proof_step_analysis["directly_related_items"]:
    print(key)

In [ ]:
proof_step_analysis["directly_related_items"]